In [1]:
import genjax
from genjax import dippl
from genjax import gensp
import jax
import adevjax

console = genjax.pretty(show_locals=True)
key = jax.random.PRNGKey(314159)

I0000 00:00:1695998673.193546   64122 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
@genjax.gen
def model(p):
    v = dippl.flip_enum(p) @ "v"
    return v


lifted = gensp.choice_map_distribution(model, genjax.select("v"), None)


def emitter(key, p):
    key, sub_key = jax.random.split(key)
    key, logpdf_key = jax.random.split(key)

    @adevjax.adev
    def flip_loss(p):
        (_, v) = lifted.random_weighted(sub_key, p)
        w = lifted.estimate_logpdf(logpdf_key, v, p)
        b = v.get_leaf_value()["v"]
        return jax.lax.cond(
            b,
            lambda _: w * 0.0,
            lambda p: w * (-p / 2.0),
            p,
        )

    return adevjax.E(flip_loss).grad_estimate(key, p)


jax.jit(emitter)(key, (0.3,))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:29                                                                                   │
│                                                                                                  │
│   26 │   return adevjax.E(flip_loss).grad_estimate(key, p)                                       │
│   27                                                                                             │
│   28                                                                                             │
│ ❱ 29 jax.jit(emitter)(key, (0.3,))                                                               │
│   30                                                                                             │
│                                                                                                  │
│ ╭─────────────────────────────────────────── locals ───────────────────────────────────────────╮ │
│ │     adevjax = <module 'adevjax' from                                                         │ │
│ │               '/home/femtomc/miniconda3/envs/py311/lib/python3.11/site-packages/adevjax/__i… │ │
│ │     console = GenJAXConsole(rich_console=<console width=115 ColorSystem.TRUECOLOR>)          │ │
│ │       dippl = <module 'genjax.dippl' from                                                    │ │
│ │               '/home/femtomc/Research/genjax/src/genjax/dippl.py'>                           │ │
│ │     emitter = <function emitter at 0x7f5a8c715620>                                           │ │
│ │        exit = <IPython.core.autocall.ZMQExitAutocall object at 0x7f5b3142f450>               │ │
│ │      genjax = <module 'genjax' from '/home/femtomc/Research/genjax/src/genjax/__init__.py'>  │ │
│ │       gensp = <module 'genjax.gensp' from                                                    │ │
│ │               '/home/femtomc/Research/genjax/src/genjax/gensp.py'>                           │ │
│ │ get_ipython = <bound method InteractiveShell.get_ipython of                                  │ │
│ │               <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7f5b313f4d10>>             │ │
│ │          In = [                                                                              │ │
│ │               │   '',                                                                        │ │
│ │               │   'import genjax\nfrom genjax import dippl\nfrom genjax import gensp\nimport │ │
│ │               jax\nimpor'+85,                                                                │ │
│ │               │   '@genjax.gen\ndef model(p):\n    v = dippl.flip_enum(p) @ "v"\n    return  │ │
│ │               v\n\n\nlifted'+580                                                             │ │
│ │               ]                                                                              │ │
│ │         jax = <module 'jax' from                                                             │ │
│ │               '/home/femtomc/miniconda3/envs/py311/lib/python3.11/site-packages/jax/__init_… │ │
│ │         key = Array([     0, 314159], dtype=uint32)                                          │ │
│ │      lifted = ChoiceMapDistribution(                                                         │ │
│ │               │   p=BuiltinGenerativeFunction(source=<function model at 0x7f5a8c7149a0>),    │ │
│ │               │   selection=HierarchicalSelection(trie=Trie(inner={'v': AllSelection()})),   │ │
│ │               │   custom_q=None                                                              │ │
│ │               )                                                                              │ │
│ │       model = BuiltinGenerativeFunction(source=<function model at 0x7f5a8c7149a0>)           │ │
│ │        open = <function open at 0x7f5b32732020>                                              │ │
│ │         Out = {}                                         